In [7]:
from adversarial_lab.core.noise_generators import AdditiveNoiseGenerator
from adversarial_lab.attacks.whitebox import WhiteBoxMisclassification
from adversarial_lab.core.preprocessing import PreprocessingFromFunction
from adversarial_lab.utils.plotting import plot_images_and_noise

from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

In [8]:
model = InceptionV3(weights='imagenet')

def preprocess(sample, *args, **kwargs):
    input = tf.cast(sample, dtype=tf.float32)
    if len(input.shape) == 2:
        input = tf.expand_dims(input, axis=-1)
        input = tf.image.grayscale_to_rgb(input)

    elif len(input.shape) == 3 and input.shape[-1] == 1:
        input = tf.image.grayscale_to_rgb(input)

    input_tensor = tf.convert_to_tensor(input, dtype=tf.float32)
    resized_image = tf.image.resize(input_tensor, (299, 299))
    batch_image = tf.expand_dims(resized_image, axis=0)
    return preprocess_input(batch_image)

pp = PreprocessingFromFunction.create(preprocess)

In [9]:
image = Image.open('data/panda.jpg')
image_array = np.array(image)

np.argmax(model.predict(pp.preprocess(image_array)), axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([388], dtype=int64)

In [10]:
attacker = WhiteBoxMisclassification(model, "cce", "adam", preprocessing=pp)

Attacking: 100%|██████████| 20/20 [01:50<00:00,  5.54s/it, Loss=5.86, Prediction=388, Prediction Confidence=0.739]


In [11]:
noise = attacker.attack(image_array, epochs=100, strategy="random", verbose=3)

Attacking: 100%|██████████| 100/100 [04:12<00:00,  2.49s/it, Loss=4.74, Prediction=388, Prediction Confidence=0.0505]

In [12]:
plot_images_and_noise(pp.preprocess(image_array), noise, attacker.noise_generator)

TypeError: plot_images_and_noise() takes 2 positional arguments but 3 were given